In [1]:
import numpy as np
import sys

train_data = np.loadtxt('./hw4_train.dat.txt')
test_data = np.loadtxt('./hw4_test.dat.txt')

In [2]:
def sign(x):
    r = np.ones(x.shape)
    for idx, value in enumerate(x):
        r[idx] = (-1 if value < 0 else 1)
    return r

def get_dataset(x):
    X = x[:, 0:x.shape[1]-1]
    y = x[:, x.shape[1]-1: x.shape[1]]
    X = np.concatenate((np.ones((X.shape[0],1)), X), axis=1)
    return X,y

X_train, y_train = get_dataset(train_data)
X_test, y_test = get_dataset(test_data)

print(X_train)
print(X_train.shape, y_train.shape)

[[1.         0.568304   0.568283  ]
 [1.         0.310968   0.310956  ]
 [1.         0.103376   0.103373  ]
 [1.         0.0531882  0.053218  ]
 [1.         0.97006    0.970064  ]
 [1.         0.0941873  0.0941707 ]
 [1.         0.655902   0.655892  ]
 [1.         0.370821   0.370839  ]
 [1.         0.558482   0.558476  ]
 [1.         0.849389   0.849383  ]
 [1.         0.796038   0.796051  ]
 [1.         0.723246   0.723252  ]
 [1.         0.571236   0.571254  ]
 [1.         0.385144   0.38512   ]
 [1.         0.877176   0.877168  ]
 [1.         0.74655    0.746552  ]
 [1.         0.0676164  0.0676087 ]
 [1.         0.0412524  0.0412649 ]
 [1.         0.851637   0.851661  ]
 [1.         0.586989   0.58698   ]
 [1.         0.661014   0.660994  ]
 [1.         0.587988   0.587968  ]
 [1.         0.257615   0.257628  ]
 [1.         0.680505   0.680485  ]
 [1.         0.895242   0.895257  ]
 [1.         0.381124   0.381139  ]
 [1.         0.314332   0.31433   ]
 [1.         0.157744   0.15

In [3]:
def get_w_reg(x, y, lambda_reg):
    # linear regression
    # w_reg = (((x_tran*x + lambda * I)_inver)* x_tran) * y
    #       (x_tran*x + lambda * I) = first_part
    #       first_part_inver * x_tran = second_part
    # w_reg = second_part * y
    first_part = np.dot(np.transpose(x), x) + lambda_reg * np.eye(x.shape[1])
    first_part_inver = np.linalg.inv(first_part)
    second_part = np.dot(first_part_inver, np.transpose(x))
    w_reg = np.dot(second_part, y)
    return w_reg

In [4]:
def get_error(x, y, w):
    return float(np.sum(sign(np.dot(x, w)) != y))/x.shape[0]

def q_13(lambda_reg = 10):
    w = get_w_reg(X_train, y_train, lambda_reg)
    e_in = get_error(X_train, y_train, w)
    e_out = get_error(X_test, y_test, w)
    return (e_in, e_out)

In [5]:
def q_1415(is_q_15 = False):
    e_in_min = 100
    e_out_min = e_in_min
    lambda_reg_min = 100
    for log_lambda in range(2, -11, -1):
        lambda_reg = 10 ** log_lambda
        e_in, e_out = q_13(lambda_reg)
        if is_q_15 is False:
            if e_in < e_in_min:
                e_in_min = e_in
                e_out_min = e_out
                lambda_reg_min = log_lambda
        else:
            if e_out < e_out_min:
                e_in_min = e_in
                e_out_min = e_out
                lambda_reg_min = log_lambda
        
    return e_in_min, e_out_min, lambda_reg_min

In [6]:
def q_161718(is_q_17=False, is_q_18 = False):
    n_train = 120
    _X_train = X_train[:n_train]
    _y_train = y_train[:n_train]
    if is_q_18:
        _X_train = X_train
        _y_train = y_train
    X_val = X_train[n_train:]
    y_val = y_train[n_train:]
    e_in_min = 100
    e_val_min = e_in_min
    e_out_min = e_val_min
    lambda_reg_min = e_out_min
    for log_lambda in range(2, -11, -1):
        lambda_reg = 10 ** log_lambda
        w_reg = get_w_reg(_X_train, _y_train, lambda_reg)
        e_in = get_error(X_train[:n_train], y_train[:n_train], w_reg)
        e_val = get_error(X_val, y_val, w_reg)
        e_out = get_error(X_test, y_test, w_reg)
        if is_q_17 is False:
            if e_in < e_in_min:
                e_in_min = e_in
                e_val_min = e_val
                e_out_min = e_out
                lambda_reg_min = log_lambda
        else:
            if e_val < e_val_min:
                e_in_min = e_in
                e_val_min = e_val
                e_out_min = e_out
                lambda_reg_min = log_lambda
    return e_in_min, e_val_min, e_out_min, lambda_reg_min

In [21]:
def get_cross_dataset(x, y, k):
    try:
        if k > 1:
            n_dataset = int(x.shape[0]/k)
            x_result = []
            y_result = []
            for idx in range(k):
                r_init = n_dataset * idx
                r_fin = r_init + n_dataset
                x_result.append(x[r_init: r_fin])
                y_result.append(y[r_init: r_fin])
            return x_result, y_result
        else:
            raise ValueError("k must greater that 1")
    except ValueError:
        raise

def q_1920():
    # k cross validatioin
    x_datasets, y_datasets = get_cross_dataset(X_train, y_train, 5)
    
    e_val_min = 100000
    lambda_reg_min = e_val_min
    
    for log_lambda in range(2, -11, -1):
        lambda_reg = 10 ** log_lambda
        current_val_idx = 0
        e_val = 0
        while current_val_idx < len(x_datasets):
            _x_train = None
            _y_train = None
            _x_val = None
            _y_val = None
            for idx, value in enumerate(x_datasets):
                if idx != current_val_idx:
                    # 拿來訓練
                    if _x_train is None:
                        _x_train = value
                        _y_train = y_datasets[idx]
                    else:
                        _x_train = np.vstack((_x_train, value))
                        _y_train = np.vstack((_y_train, y_datasets[idx]))
                else:
                    # 拿來validation
                    _x_val = value
                    _y_val = y_datasets[idx]
            w_reg = get_w_reg(_x_train, _y_train, lambda_reg)
            # 算每一輪e_val的加總
            e_val += get_error(_x_val, _y_val, w_reg)  
            current_val_idx += 1
        # 然後再取平均
        e_val_avg = e_val/len(x_datasets)
        
        # 找出最小的
        if e_val_avg < e_val_min:
            e_val_min = e_val_avg
            lambda_reg_min = log_lambda
    return e_val_min, lambda_reg_min

In [8]:
print("q13")
e_in, e_out = q_13()
print(e_in, e_out)
# q13的答案 e_in = 0.05, e_out = 0.045
print("")
print("q14")
e_in, e_out, lambda_reg = q_1415()
print(lambda_reg, e_in, e_out)
# q_14 -> lamba = -8 e_in = 0.015, e_out = 0.02
print("")
print("q15")
e_in, e_out, lambda_reg = q_1415(True)
print(lambda_reg, e_in, e_out)
# q_15 -> lamba = -7 e_in =. 0.03, e_out = 0.015

q13
0.05 0.045

q14
-8 0.015 0.02

q15
-7 0.03 0.015


In [9]:
print("q16")
e_in, e_val, e_out, lambda_reg = q_161718()
print(lambda_reg, e_in, e_val, e_out)
# q_16 -> lamba = -8 e_in =. 0.0, e_val = 0.05, e_out = 0.025
print("")
print("q17")
e_in, e_val, e_out, lambda_reg = q_161718(True)
print(lambda_reg, e_in, e_val, e_out)
# q_17 -> lamba = 0 e_in =. 0.03, e_val = 0.038, e_out = 0.028
print("")
print("q18")
e_in, e_val, e_out, lambda_reg = q_161718(True, True)
print(e_in, e_out)
# q_18 -> e_in =. 0.035, e_out = 0.02

q16
-8 0.0 0.05 0.025

q17
0 0.03333333333333333 0.0375 0.028

q18
0.03333333333333333 0.016


In [22]:
print("q19")
e_cv, lambda_reg = q_1920()
print(lambda_reg, e_cv)
# q_19 -> lambda = -8, e_cv = 0.03
print("")
print("q20")
e_in, e_out = q_13(10**lambda_reg)
print(e_in, e_out)
# q_20 -> e_in = 0.015, e_out = 0.02

q19
-8 0.03

q20
0.015 0.02
